In [1]:
#import basic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#import model libraries
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject


In [4]:
#import dataset
df = pd.read_csv(r'C:\Users\Rengarajan\student_performance_ml\notebook\data/StudentsPerformance.csv')

In [5]:
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [6]:
#preparing X dataset
X = df.drop(columns=['math score'],axis=1)
X.head(2)

,gender,race/ethnicity,parental level of education,lunch,test preparation course,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88


In [7]:
#preparing y dataset
y = df['math score']
y.head(2)

0    72
1    69
Name: math score, dtype: int64

In [8]:
#creating a pipeling which will do one hot encoding for categorical features, standardization for numeric features
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

#getting numerical featues in X dataset
num_features = X.select_dtypes(exclude='object').columns

#getting categorical featues in X dataset
cat_features = X.select_dtypes(include='object').columns

num_transformer = StandardScaler()
cat_transformer = OneHotEncoder()

preprocessor = ColumnTransformer([('OneHotEncoder', cat_transformer, cat_features),
                                 ('StandardScaler', num_transformer, num_features),])

In [9]:
X = preprocessor.fit_transform(X)
X.shape

(1000, 19)

In [10]:
#separate the data into train and test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((800, 19), (200, 19))

In [13]:
#create fun to give all metrics after model training
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    #mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_value = r2_score(true, predicted)
    return mae,rmse,r2_value

In [14]:
#model input and training the models

models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'KNN regressor': KNeighborsRegressor(),
    'Decision Tree regression': DecisionTreeRegressor(),
    'Random forest regression': RandomForestRegressor(),
    'XGBoost Regression': XGBRegressor(),
    'catboost regression': CatBoostRegressor(verbose=False),
    'adaboost regression': AdaBoostRegressor()
}
model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)     #training the model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Evaluate Train and Test dataset
    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model performance for Training set')
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)

    print('='*35)
    print('\n')



Linear Regression
Model performance for Training set
- Mean Absolute Error: 4.2715
- Root Mean Squared Error: 5.3293
- R2 Score: 0.8740
----------------------------------
Model performance for Test set
- Mean Absolute Error: 4.2222
- Root Mean Squared Error: 5.4252
- R2 Score: 0.8790


Ridge Regression
Model performance for Training set
- Mean Absolute Error: 4.2650
- Root Mean Squared Error: 5.3233
- R2 Score: 0.8743
----------------------------------
Model performance for Test set
- Mean Absolute Error: 4.2111
- Root Mean Squared Error: 5.3904
- R2 Score: 0.8806


Lasso Regression
Model performance for Training set
- Mean Absolute Error: 5.2063
- Root Mean Squared Error: 6.5938
- R2 Score: 0.8071
----------------------------------
Model performance for Test set
- Mean Absolute Error: 5.1579
- Root Mean Squared Error: 6.5197
- R2 Score: 0.8253


KNN regressor
Model performance for Training set
- Mean Absolute Error: 4.5168
- Root Mean Squared Error: 5.7079
- R2 Score: 0.8555
---------

In [15]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)


,Model Name,R2_Score
1,Ridge Regression,0.880593
0,Linear Regression,0.879046
8,adaboost regression,0.854002
5,Random forest regression,0.853224
7,catboost regression,0.851632
2,Lasso Regression,0.825320
6,XGBoost Regression,0.821589
3,KNN regressor,0.783813
4,Decision Tree regression,0.740546
